In [1]:
# Import our dependencies and libraries
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from sklearn.linear_model import LinearRegression

# Read in our housing data
housing_data_df= pd.read_csv("../Resources/City_Zhvi_AllHomes_for_ML_Linear_Regression.csv",parse_dates=True)
housing_data_df

,Unnamed: 0,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,CountyName,1996-01-31,...,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31
0,0,6181,0,New York,City,NY,NY,New York-Newark-Jersey City,Queens County,196258.0,...,659421.0,659007.0,658239.0,656925.0,655613.0,654394.0,653930.0,653901.0,653565.0,652307.0
1,1,12447,1,Los Angeles,City,CA,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,185649.0,...,712660.0,713807.0,715688.0,718245.0,721896.0,725180.0,730358.0,735910.0,744137.0,752508.0
2,2,39051,2,Houston,City,TX,TX,Houston-The Woodlands-Sugar Land,Harris County,93518.0,...,186844.0,187464.0,188070.0,188496.0,189125.0,189612.0,190179.0,190395.0,190938.0,191907.0
3,3,17426,3,Chicago,City,IL,IL,Chicago-Naperville-Elgin,Cook County,130920.0,...,248372.0,248646.0,248725.0,248483.0,248278.0,248090.0,248029.0,248220.0,248599.0,249152.0
4,4,6915,4,San Antonio,City,TX,TX,San Antonio-New Braunfels,Bexar County,94041.0,...,182732.0,183350.0,183930.0,184846.0,185490.0,186244.0,186420.0,186962.0,187129.0,187718.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27325,27325,48550,29449,Winton,City,MN,MN,Duluth,Saint Louis County,NaN,...,70929.0,70176.0,68979.0,68537.0,68948.0,69481.0,69423.0,69355.0,69654.0,70399.0
27326,27326,51821,29449,Eastabuchie,City,MS,MS,Laurel,Jones County,NaN,...,77737.0,78461.0,79138.0,79930.0,79683.0,79838.0,80272.0,80969.0,81871.0,82284.0
27327,27327,232607,29449,Dean,City,TX,TX,Wichita Falls,Clay County,NaN,...,178872.0,179976.0,180654.0,181188.0,181613.0,182101.0,182424.0,182764.0,182639.0,182361.0
27328,27328,6638,29449,Pulaski,City,GA,GA,NaN,Candler County,NaN,...,97356.0,97757.0,98014.0,98225.0,98303.0,98613.0,98802.0,99227.0,99699.0,100464.0


In [ ]:
#Make a top city dataframe based on size rank of each city
topcity = housing_data_df[['SizeRank', 'RegionName']].copy()
topcity

In [ ]:
#Reduce the top city dataframe to the top 60 cities by size rank
topcity = topcity.loc[0:59,'RegionName']

In [ ]:
#Covert city dataframe to list
topcity.values.tolist()

In [ ]:
## start dropping unneeded columns
## dropped RegionID, State, Metro, CountyName 
housing_data_df=housing_data_df.drop(housing_data_df.iloc[:, 0:3],axis = 1)
housing_data_df.head()

In [ ]:
## drop remaining unneeded columns
## dropped all months before January 2010
housing_data_df=housing_data_df.drop(housing_data_df.iloc[:, 1:174],axis = 1)
housing_data_df.head()

In [ ]:
#Dropping all but first value in RegionName. City names appear more than once if there are cities in diffrent states 
# with the same name. Since we are looking for top cities we are ok to drop all occurances after the first.
housing_data_df.drop_duplicates(subset=['RegionName'], inplace=True)
housing_data_df

In [ ]:
#keep only rows with the cities in our top cities by size
housing_topcity_df = housing_data_df[housing_data_df['RegionName'].isin(topcity)]
housing_topcity_df

In [ ]:
## transpose dates to rows
housing_topcity_df=housing_topcity_df.transpose()
housing_topcity_df.head()

In [ ]:
#move first row to column names
housing_topcity_df.columns = housing_topcity_df.iloc[0]
housing_topcity_df

In [ ]:
#remove first row shwowing city name
housing_topcity_df = housing_topcity_df.iloc[1: , :]
housing_topcity_df

In [ ]:
# Reset index
housing_topcity_df = housing_topcity_df.reset_index()
housing_topcity_df.head()

In [ ]:
#Rename date column to date
housing_topcity_df.rename(columns={ housing_topcity_df.columns[0]: "Date" })

In [ ]:
#iterate over columns to create a linear regression model for each city for the date range of 2010 to 2020
#create an empty list to hold our house price predictions once generated

housing_predictions = []

#loop through each column after the date column 

for column in housing_topcity_df.columns[1:]:
    
    
    X = housing_topcity_df.index.values.reshape(-1, 1)   

    #convert X values to float type
    x = X.astype(float)

    Y = housing_topcity_df[column].values.reshape(-1, 1)   

#convert Y value to float type
    y = Y.astype(float)   

    linear_regressor = LinearRegression()  # create object for the class
    linear_regressor.fit(x, y)  # perform linear regression
    Y_pred = linear_regressor.predict(X)  # make predictions
    
    #predict the value in 6 months from last data point - this will be equal to the average of the next year
    x_new = np.arange(129).reshape((-1, 1)) 
    
    #set the list to append only the last value predcited by the model which is the value 6 months from last actual data point
    y_new = linear_regressor.predict(x_new)
    y_new[-1][-1]
    housing_predictions.append([column, y_new[-1][-1]])

In [ ]:
#View the list of housing predictions generated for 6 months from last actual data point
housing_predictions

In [ ]:
#convert regrresssion prediction results to a dataframe
future_prediction_df = pd.DataFrame(housing_predictions)
future_prediction_df

In [ ]:
#renaming future price predcition column
future_prediction_df.columns = ['City', 'Future Price']
future_prediction_df

In [ ]:
#changing Future Price column to an integer
future_prediction_df['Future Price'] = future_prediction_df['Future Price'].astype(int)
future_prediction_df

In [ ]:
#convert result to CSV file
next_month_prediction_df.to_csv('../Resources/future_housing_prediction.csv')